# Rat model minimal example


## Model objectives

This minimal example sets out to solve the following model objectives:

 - The rat prefers the moderate solution to the strong solution under salt depletion; it may approach the strong solution sometimes, but less often than the moderate solution
 - The rat prefers the moderate solution to the strong solution under salt satiation (i.e., finds it less aversive)
 - The rate will take the strong solution under salt depletion if it has no other option
 - Something representing reward signal will fire for a sugar solution but not a salt solution when there is a need for salt; however, when there is a need for salt, the reward signal will fire regardless

## Objectives 1 and 2: multiple stimulus features

We can specify the basic model we need to model salt like:

\begin{align}\label{eq:BasicSaltModelZBVector}
\tilde r(\mathbf{r},\mathbf{\kappa})=& \mathbf{\kappa}_{\text{Na}} \mathbf{r}_{\text{Na}}+\mathbf{\kappa}_{h} \mathbf{r}_{h}
\end{align}

## Objectives 3: environmental cues
To meet Objective 3 above--to show the rat will drink the salty solution if it is salt-depleted and is the only one available--we first need to assume a different weighting of hydration to deprivation. For instance, we can do that by increasing the deprivation/interoceptive state value $\kappa_{\text{Na}}$ from 1 to 1.5.

Now, in order to make this aspect of the model explicit we need to introduce an extra vector describing environmental cues that elicit responses, $\mathbf{c}$. Following the S-R-O expectancy valence account given in \textcite{zhang2009neural}, the extra parameter describes environmental features eliciting a response. 

Using the new model with our vector representing environmental cues $\mathbf{c}$, we can predict behavior where there is no cue for a moderate salt solution (Note the use of elementwise multiplication between the matrices using the symbol $\circ$):

$$
\begin{align}\label{eq:WithCues}
\tilde r(\mathbf{r},\mathbf{\kappa})= & \mathbf{\kappa}_{\textrm{Na}} \mathbf{r}_\textrm{Na}\mathbf{c}+\mathbf{\kappa}_{h} \mathbf{r}_{h}\mathbf{c} % \\
% = & 1.5 \times \begin{bmatrix}0.5 \\ 1.0\end{bmatrix}\circ \begin{bmatrix}0.0 \\ 1.0\end{bmatrix} + 1 \times \begin{bmatrix}0.0 \\ -1.0\end{bmatrix}\circ \begin{bmatrix}0.0 \\ 1.0\end{bmatrix} \nonumber \\
% = & \begin{bmatrix}0.0 \\ 0.5\end{bmatrix} \nonumber
\end{align}
$$

Here, I've modeled $\mathbf{c}$ as a single vector without a subscript to indicate that we do not expect the cue value to systematically differ for each interoceptive value. This differs from the other documents produced--there, I'd included a subscript, more because I didn't think it through than any other reason.

## Objective 4: Sugar

The model in Equation~\ref{eq:BasicSaltModelZBVector} can be extended slightly to account for the evidence discussed in the sugar example in Tindell et al. (2009):

\begin{equation}\label{eq:SaltSugarModelTindell}
\tilde r(\mathbf{r},\mathbf{\kappa})=\kappa_{\text{Na}} r_{\text{Na}}+\kappa_{h} r_{h} + \kappa_{\text{Glc}} r_{\text{Glc}}
\end{equation}

## Combined Model

Combining the models presented above, we obtain the model 

\begin{align}
\begin{split}\label{eq:SaltSugarModelWithCue}
\tilde r(\mathbf{r},\mathbf{\kappa})= & \kappa_{\text{Na}} r_{\text{Na}}\mathbf{c}+\kappa_{h} r_{h}\mathbf{c} + \kappa_{\text{Glc}} r_{\text{Glc}}\mathbf{c}
\end{split}
\end{align}

We can generalize this further by describing an arbitrary number of cue-stimulus-interoception multiplicative terms:

\begin{align}
\tilde r(\mathbf{r},\mathbf{\kappa},\mathbf{c})= & \kappa_{1} \mathbf{r}_{1}\mathbf{c}+\dots+\kappa_{\text{N}} \mathbf{r}_{\text{N}}\mathbf{c}
\end{align}

We can then write a python function to model this equation, and then demonstrate how it handles each situation described above.

The function ```multiplicative_term``` forms a python dictionary representing a term with kappa value, r array, and cue array. The function ```SaltSugarModelWithCue``` is simply a python implementation of the $r(\mathbf{r},\mathbf{\kappa},\mathbf{c})$ function nabove. It calls ```SaltSugarModelWithCue_SingleTerm```, which is written separately so that it can be used directly to show the working in the equation below.

In [1]:
from minimal_example_interface import *
import numpy as np

def multiplicative_term(kappa_val,r_array,cue_array):
    assert(type(kappa_val)==float or type(kappa_val)==int)
    assert(type(r_array)==np.ndarray)
    assert(type(cue_array)==np.ndarray)
    
    return({"kappa": kappa_val, "r": r_array,"cue": cue_array})

def SaltSugarModelWithCue_SingleTerm(term):
    return term['kappa']*term['r']*term['cue']

def SaltSugarModelWithCue(term_list):
    return sum([SaltSugarModelWithCue_SingleTerm(term) for term in term_list])




We can then create variables to represent each term and then calculate their value:

In [2]:
term_Na=multiplicative_term(1,np.array([0.5,1.0,0.0]),np.array([1.0,1.0,1.0]))
term_h=multiplicative_term(1,np.array([0.0,-1.0,0.0]),np.array([1.0,0.5,1.0]))
term_Glc=multiplicative_term(1,np.array([0.0,0.0,1.0]),np.array([1.0,1.0,.7171]))



We can write this out mathematically:

In [16]:
from IPython.display import Markdown

# main_equation_text = """
#     $$
#     \\begin{{align}}
#     \\begin{{split}}
#     \\tilde r(\mathbf{{r}},\mathbf{{\kappa}},\mathbf{{c}})= & 
#     \kappa_{{\\text{{Na}}}} \mathbf{{r}}_{{\\text{{Na}}}}\mathbf{{c}} 
#     & + \kappa_{{h}} \mathbf{{r}}_{{h}}\mathbf{{c}}
#     & + \kappa_{{\\text{{Glc}}}} \mathbf{{r}}_{{\\text{{Glc}}}}\mathbf{{c}} \\\\
#      = & {Na_term} & + {h_term} & + {Glc_term} \\\\
#      = & {Na_val} & + {h_val} & + {Glc_val} \\\\
#      = & {function_val}
#     \end{{split}}
#     \end{{align}}
#     $$
#     """.format(Na_term=krc_term_to_latex(term_Na), 
#                h_term=krc_term_to_latex(term_h),
#                Glc_term=krc_term_to_latex(term_Glc),
#               Na_val=ndarray_latex_format(SaltSugarModelWithCue_SingleTerm(term_Na)),
#               h_val=ndarray_latex_format(SaltSugarModelWithCue_SingleTerm(term_h)),
#               Glc_val=ndarray_latex_format(SaltSugarModelWithCue_SingleTerm(term_Glc)),
#               function_val=ndarray_latex_format(SaltSugarModelWithCue([term_Na,term_h,term_Glc])))
# def get_main_equation_text():
#     return main_equation_text
equation_markdown_text="""
$$
\\begin{{align}}
\\begin{{split}}
\\tilde r(\mathbf{{r}},\mathbf{{\kappa}},\mathbf{{c}})= & 
\kappa_{{\\text{{Na}}}} \mathbf{{r}}_{{\\text{{Na}}}}\mathbf{{c}} 
& + \kappa_{{h}} \mathbf{{r}}_{{h}}\mathbf{{c}}
& + \kappa_{{\\text{{Glc}}}} \mathbf{{r}}_{{\\text{{Glc}}}}\mathbf{{c}} \\\\
 = & {Na_term} & + {h_term} & + {Glc_term} \\\\
 = & {Na_val} & + {h_val} & + {Glc_val} \\\\
 = & {function_val}
\end{{split}}
\end{{align}}
$$
""".format(Na_term=krc_term_to_latex(term_Na), 
               h_term=krc_term_to_latex(term_h),
               Glc_term=krc_term_to_latex(term_Glc),
              Na_val=ndarray_latex_format(SaltSugarModelWithCue_SingleTerm(term_Na)),
              h_val=ndarray_latex_format(SaltSugarModelWithCue_SingleTerm(term_h)),
              Glc_val=ndarray_latex_format(SaltSugarModelWithCue_SingleTerm(term_Glc)),
              function_val=ndarray_latex_format(SaltSugarModelWithCue([term_Na,term_h,term_Glc])))

Markdown(equation_markdown_text)


$$
\begin{align}
\begin{split}
\tilde r(\mathbf{r},\mathbf{\kappa},\mathbf{c})= & 
\kappa_{\text{Na}} \mathbf{r}_{\text{Na}}\mathbf{c} 
& + \kappa_{h} \mathbf{r}_{h}\mathbf{c}
& + \kappa_{\text{Glc}} \mathbf{r}_{\text{Glc}}\mathbf{c} \\
 = & 1 \times \begin{bmatrix}0.5 \\ 1.0 \\ 0.0\end{bmatrix}\circ \begin{bmatrix}1.0 \\ 1.0 \\ 1.0\end{bmatrix} & + 1 \times \begin{bmatrix}0.0 \\ -1.0 \\ 0.0\end{bmatrix}\circ \begin{bmatrix}1.0 \\ 0.5 \\ 1.0\end{bmatrix} & + 1 \times \begin{bmatrix}0.0 \\ 0.0 \\ 1.0\end{bmatrix}\circ \begin{bmatrix}1.0 \\ 1.0 \\ 0.72\end{bmatrix} \\
 = & \begin{bmatrix}0.5 \\ 1.0 \\ 0.0\end{bmatrix} & + \begin{bmatrix}0.0 \\ -0.5 \\ 0.0\end{bmatrix} & + \begin{bmatrix}0.0 \\ 0.0 \\ 0.72\end{bmatrix} \\
 = & \begin{bmatrix}0.5 \\ 0.5 \\ 0.72\end{bmatrix}
\end{split}
\end{align}
$$


You can play with the values yourself here or use the presets to see how values change.


In [4]:
display(widgets.HBox([bDeprivationPreset,bSatiationPreset,tbSugarPresence]))


### Interoception ($\kappa$)

In [5]:
widgets.HBox(ftKappaVals)


### Expected reward ($\tilde r$)

In [6]:
widgets.HBox(ftRewardVals)


### Cue Accessibility ($\mathbf{c}$)

In [7]:
widgets.VBox(ftCueVals)

In [8]:
bResetAll_on_click(0)

In [9]:
#display(widgets.VBox([
    #widgets.HBox([bDeprivationPreset,bSatiationPreset,tbSugarPresence]),
    #widgets.HBox(ftKappaVals),
#    widgets.HBox([widgets.HBox(ftRewardVals),widgets.VBox(ftCueVals)]),
#    display(main_equation)]))

def f7(myb):
    display(main_equation)

#get all the controls in the widget
#full_control_list=ftKappaVals+[sln for driveset in ftRewardVals[1:] for sln in driveset.children[1:]]+ftCueVals

#need to convert it into a bloody dictionary.
#equation_out = widgets.interactive_output(f7, full_control_list.to)

math_widget=widgets.HTMLMath(value=get_main_equation_text())
def myb_on_click():
    print("clicking")
    update_equation()
    math_widget=widgets.HTMLMath(value=get_main_equation_text())
#myb.on_click(myb_on_click)

display(widgets.VBox([
    widgets.HBox([bDeprivationPreset,bSatiationPreset,tbSugarPresence,myb]),
    widgets.HBox(ftKappaVals),
    widgets.HBox([widgets.HBox(ftRewardVals),widgets.VBox(ftCueVals)]),
    math_widget]))

NameError: name 'myb' is not defined

In [ ]:
full_control_list